In [1]:
N_JOBS = 6
N_MACHINES = 4

In [2]:
import random

def generateData(seed=7406):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(7406)

In [4]:
PROC

[[5, 8, 5, 4],
 [6, 6, 6, 4],
 [5, 5, 7, 2],
 [7, 7, 1, 4],
 [9, 3, 4, 8],
 [8, 7, 2, 7]]

In [5]:
import pulp

In [6]:
def val(x):
    return int(pulp.value(x))

In [7]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

In [8]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [9]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [10]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [11]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

In [12]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [13]:
print(Wait)

{0: {0: WAIT_0_0, 1: WAIT_0_1, 2: WAIT_0_2, 3: WAIT_0_3}, 1: {0: WAIT_1_0, 1: WAIT_1_1, 2: WAIT_1_2, 3: WAIT_1_3}, 2: {0: WAIT_2_0, 1: WAIT_2_1, 2: WAIT_2_2, 3: WAIT_2_3}, 3: {0: WAIT_3_0, 1: WAIT_3_1, 2: WAIT_3_2, 3: WAIT_3_3}, 4: {0: WAIT_4_0, 1: WAIT_4_1, 2: WAIT_4_2, 3: WAIT_4_3}, 5: {0: WAIT_5_0, 1: WAIT_5_1, 2: WAIT_5_2, 3: WAIT_5_3}}


In [14]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

In [15]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [16]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [17]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [18]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [20]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [21]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

In [22]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [23]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

In [24]:
PROC

[[5, 8, 5, 4],
 [6, 6, 6, 4],
 [5, 5, 7, 2],
 [7, 7, 1, 4],
 [9, 3, 4, 8],
 [8, 7, 2, 7]]

In [25]:
print(jobSequence())

[0, 1, 4, 5, 3, 2]


In [26]:
int(pulp.value(prob.objective))

52

In [27]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    7 |            | Idle:   16 |            | Idle:   21 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   7 |            | Start:  16 |            | Start:  21 |
| Job:  0 | Wait:    0 | Proc:    5 | Wait:    2 | Proc:    8 | Wait:    1 | Proc:    5 | Wait:    0 | Proc:    4 |
|         |            | Stop:    5 |            | Stop:   15 |            | Stop:   21 |            | Stop:   25 |
|---------|------------|------------|------------|------------|---------

In [28]:
import random

def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [29]:
PROC = generateData(7406)

In [30]:
PROC

[[5, 8, 5, 4],
 [6, 6, 6, 4],
 [5, 5, 7, 2],
 [7, 7, 1, 4],
 [9, 3, 4, 8],
 [8, 7, 2, 7]]

## Greedy Algorithm

In [31]:
import random


def val(x):
    return int(x)

def greedy_algorithm(PROC):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])
    
    # Initialize scheduling variables
    Idle = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Start = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Stop = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    
    # Initialize machine availability times
    machine_times = [0] * N_MACHINES
    
    # Schedule each job
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            if m == 0:
                Start[j][m] = machine_times[m]
            else:
                Start[j][m] = max(machine_times[m], Stop[j][m-1])
            
            Idle[j][m] = Start[j][m] - (machine_times[m] if m == 0 else Stop[j][m-1])
            Stop[j][m] = Start[j][m] + PROC[j][m]
            machine_times[m] = Stop[j][m]
    
    return Idle, Start, Stop, PROC

In [32]:
def schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC):
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Idle[j][m]):4d} | " \
                   f"Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

Idle, Start, Stop, PROC = greedy_algorithm(PROC)
print("Greedy Algorithm Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC))

Greedy Algorithm Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   5 |            | Start:  13 |            | Start:  18 |
| Job:  0 | Wait:    0 | Proc:    5 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    5 | Wait:    0 | Proc:    4 |
|         |            | Stop:    5 |            | Stop:   13 |            | Stop:   18 |            | Stop:   22 |
|---------|

In [33]:
stop_times = [
    [5, 13, 18, 22],
    [11, 19, 25, 29],
    [16, 24, 32, 34],
    [23, 31, 33, 38],
    [32, 35, 39, 47],
    [40, 47, 49, 56]
]

def calculate_overall_processing_time(stop_times):
    overall_processing_time = max(max(times) for times in stop_times)
    return overall_processing_time

overall_processing_time = calculate_overall_processing_time(stop_times)

print("Overall Processing Time:", overall_processing_time)


Overall Processing Time: 56


## Stochastic Opimisation algorithm (Simulated Annealing algorithm)

In [34]:
import random
import math

def calculate_makespan(job_order, PROC, N_MACHINES):
    # Initialize completion times for machines
    machine_times = [0] * N_MACHINES
    
    for job in job_order:
        # Start the first job at the completion time of the previous job on the first machine
        current_start_time = machine_times[0]
        for machine in range(N_MACHINES):
            if machine == 0:
                # Job starts after the previous job finished on this machine
                machine_times[machine] += PROC[job][machine]
            else:
                # Job starts after both the previous job on this machine and this job on the previous machine
                machine_times[machine] = max(machine_times[machine], machine_times[machine - 1]) + PROC[job][machine]
    return machine_times[-1]

def initial_solution(N_JOBS):
    return list(range(N_JOBS))

def get_neighbor(solution):
    # Swap two random elements
    a, b = random.sample(range(len(solution)), 2)
    solution[a], solution[b] = solution[b], solution[a]
    return solution

def simulated_annealing(PROC, N_JOBS, N_MACHINES, temp=1000, cooling_rate=0.99, stopping_temp=1):
    current_solution = initial_solution(N_JOBS)
    current_cost = calculate_makespan(current_solution, PROC, N_MACHINES)
    while temp > stopping_temp:
        new_solution = get_neighbor(current_solution[:])
        new_cost = calculate_makespan(new_solution, PROC, N_MACHINES)
        if new_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - new_cost) / temp):
            current_solution, current_cost = new_solution, new_cost
        temp *= cooling_rate
    return current_solution, current_cost

# Set a seed for reproducibility
random.seed(3546)
best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)
print("Best solution:", best_solution)
print("Best cost (makespan):", best_cost)


Best solution: [3, 1, 2, 4, 0, 5]
Best cost (makespan): 56


In [35]:
def calculate_schedule(job_order, PROC, N_JOBS, N_MACHINES):
    # Arrays to store Idle times, Start times, and Stop times for each job on each machine
    Idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Start = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Stop = [[0] * N_MACHINES for _ in range(N_JOBS)]

    #Track the end times of the last job on each machine
    machine_end_times = [0] * N_MACHINES

    #Process each job as per the job_order
    for job in job_order:
        for m in range(N_MACHINES):
            if m == 0:
                # First machine: Job starts as soon as the previous job finished
                Start[job][m] = machine_end_times[m]
            else:
                # Subsequent machines: Job starts after both the previous machine has started this job and the same machine has finished the previous job
                Start[job][m] = max(Stop[job][m-1], machine_end_times[m])

            # Processing time added to the start time to determine when it stops
            Stop[job][m] = Start[job][m] + PROC[job][m]

            # Calculate Idle time
            if m == 0:
                Idle[job][m] = 0  # First machine has no idle time for the first job
            else:
                Idle[job][m] = Start[job][m] - machine_end_times[m]

            # Update the end time for this machine
            machine_end_times[m] = Stop[job][m]

    return Idle, Start, Stop

best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)

Idle, Start, Stop = calculate_schedule(best_solution, PROC, N_JOBS, N_MACHINES)

print("Simulated Annealing Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC))


Simulated Annealing Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    6 |            | Idle:    3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   9 |            | Start:  14 |            | Start:  22 |            | Start:  27 |
| Job:  0 | Wait:    0 | Proc:    5 | Wait:    2 | Proc:    8 | Wait:    6 | Proc:    5 | Wait:    3 | Proc:    4 |
|         |            | Stop:   14 |            | Stop:   22 |            | Stop:   27 |            | Stop:   31 |
|-------

In [36]:
def calculate_overall_processing_time(Stop):
    max_stop_time = 0
    for job_stops in Stop:
        job_max = max(job_stops)
        if job_max > max_stop_time:
            max_stop_time = job_max
    return max_stop_time

Stop = [
    [14, 22, 27, 31],
    [20, 28, 34, 38],
    [33, 40, 47, 49],
    [40, 47, 48, 53],
    [9, 12, 16, 24],
    [28, 35, 37, 45]
]

overall_processing_time = calculate_overall_processing_time(Stop)
print("Overall Processing Time:", overall_processing_time)


Overall Processing Time: 53
